## PRÁCTICA TCP. SERVIDORES

Alejandro Manuel López Gómez

### PRÁCTICA TCP 1.1
Probar y analizar el funcionamiento de una sencilla aplicación cliente/servidor TCP donde el
servidor recibe un mensaje en el que el cliente se presenta y acto seguido el servidor le
responde con un mensaje de bienvenida. El servidor debe permanecer a la escucha de más
clientes hasta que reciba un mensaje de “EXIT” de alguno de ellos, en este caso, el servidor
envía un mensaje de despedida “Bye” y se desconecta.

In [1]:
import socket
import sys

# Creacion objeto socket, de la familia AF_INET, en funcion de la familia del socket,
# son necesarios pasar más parametros o la dirección del servidor en un formato distinto.
# El este caso para la familia AF_INET solo es necesaria la tupla (IP, puerto).
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Asociación al socket creado anteriormente una dirección y puerto
sock.bind(("127.0.0.1",6780))

# El servidor queda a la escucha de conexiones. El argumento pasado se
# denomina "backlog" y es el máximo número de conexiones en cola permitidas
sock.listen(1)

while True:
    print("Waiting for clients...")
    
    # Al recibirse una conexión al socket, este comando devuelve un objetivo connection utilizado
    # para la recepción y transmisión de información y la dirección del cliente como (IP, puerto)
    conn, client_address = sock.accept()
    
    # Esta instrucción bloquea el código hasta que se reciba una respuesta
    # recv(): Comunicaciones orientadas a conexión, devuelve solo los datos (TCP)
    # recvfrom(): Comunicaciones no orientadas a conexión, devuelve los datos y la dirección del emisor (UDP)
    # El parámetro pasado es el máximo tamaño de paquete en bytes permitido
    data = conn.recv(1024).decode()
    
    if (str(data) == "EXIT"):
        print("Received EXIT Command from client")
        print("Bye")
        
        # Envío de mensaje de despedida a cliente
        conn.sendall("Bye Client".encode())
        
        # Finalización de la conexión
        conn.close()
        break
    else:
        print("Received data from client: " + str(data))
        conn.sendall("Hi TCP Client".encode())

Waiting for clients...
Received data from client: Hola servidor
Waiting for clients...
Received EXIT Command from client
Bye


### PRÁCTICA TCP 1.2
Tanto el cliente como el servidor sólo puedan recibir paquetes de un
máximo de 20 bytes con la instrucción socket.recv(20). Esto implica que el código Python debe
iterar hasta que se ha recibido el mensaje completo.

In [ ]:
import socket
import sys

# En este caso solo se permiten mensajes de máximo 20 bytes. Aunque la MTU se especifique a 20 bytes,
# La librería socket implementa una cola que permite la recepción de mensajes más largos.
# Por tanto, no se elimina el mensaje si es más largo, si no que la recepción se realiza de 20 en 20 bytes.
MTU = 20

# Parámetro empleado como aviso de llegada de comando EXIT al bucle más externo, lo que
# permite cerrar la conexión detener el script
flag = 0

msg = "Hello Client, this is a long message that you will received in chunks of 20 bytes"
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind(("127.0.0.1",6780))
sock.listen(1)

while True:
    print("Waiting for clients...")
    conn, client_address = sock.accept()
    print("Connection received from client " + str(client_address))
    
    try:
        while True:
            data = conn.recv(MTU).decode()

            if(str(data) == "EXIT"):
                print("Received EXIT command from client")
                conn.sendall("Bye".encode())
                
                # Como se ha recibido EXIT, se pone flag a 1
                flag = 1
                break
            
            # Se imprime el mensaje en bloques de longitud MTU bytes
            elif (len(data) == MTU):
                print("Client> " + str(data))
            else:
                print("Client> " + str(data))
                conn.sendall(msg.encode())
                break

        if flag == 1:
            # Finalización de la conexión
            conn.close()
            break
    except ConnectionResetError:
        # En caso de interrupción de la conexión por parte del cliente,
        # se cierra la conexión y finaliza el programa
        print("Lost connection")
        conn.close()
        break

Waiting for clients...
Connection received from client ('127.0.0.1', 56307)
Client> Hola sercidor, esto 
Client> es un mensaje muy pe
Client> ro que muy largo
Waiting for clients...


### PRÁCTICA TCP 1.3
El servidor debe devolver la hora al cliente. El cliente debe recibirlos, calcular la
diferencia respecto a la hora local y almacenar la diferencia de tiempos en una lista. Al final, se
debe proporcionar el tiempo necesario para realizar 100.000 consultas de tiempo y
proporcionar el valor promedio de las diferencias de tiempo entre la hora del servidor y la del
cliente.

In [ ]:
import datetime
import socket
import sys

MTU = 1024
flag = 0

msg = "Hello Client, this is a long message that you will received in chunks of 20 bytes"
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind(("127.0.0.1",6780))
sock.listen(1)

while True:
    print("Waiting for clients...")
    conn, client_address = sock.accept()
    print("Connection received from client " + str(client_address))
    
    try:
        while True:
            data = conn.recv(MTU).decode()

            if(str(data) == "EXIT"):
                print("Received EXIT command from client")
                conn.sendall("Bye".encode())
                flag = 1
                break
                
            # Adición del comando TIME, que devuelve un objeto datetime casteado a string para que sea
            # compatible con la instrucción .encode(). El cliente deberá recibir este string y parsearlo
            # de nuevo a un objeto datetime para su correcta interpretación
            elif (str(data) == "TIME"):
                time = datetime.datetime.now()
                conn.sendall(str(time).encode())

        if flag == 1:
            conn.close()
            break
            
    except ConnectionResetError:
        print("Lost connection")
        conn.close()
        break

Waiting for clients...
Connection received from client ('127.0.0.1', 56417)


### PRÁCTICA TCP 2.1
Codificar un programa cliente/servidor que permita la recepción desde el servidor, de un
fichero solicitado por un cliente. Realmente se envía el contenido del fichero para que el cliente cree un fichero nuevo y escriba en él el contenido enviado por el servidor

In [1]:
import datetime
import socket
import sys

MTU = 1024
flag = 0

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind(("127.0.0.1",6780))
sock.listen(1)

while True:
    print("Waiting for clients...")
    conn, client_address = sock.accept()
    print("Connection received from client " + str(client_address))
    
    try:
        while True:
            data = str(conn.recv(MTU).decode())
        
            # Adición de la instrucción GET. Se examinan los tres primeros caracteres en busca de la
            # palabra clave, a continuación (suponiendo que se ha dejado un espacio) se obtiene el nombre
            # del fichero pedido por el cliente.
            # Finalmente el script abre el fichero, lee el contenido y lo envía como un string al cliente
            if (data[0:3] == "GET"):
                print("Received GET command from client")
                
                filename = data[4:]
                with open(filename) as f:
                    contents = f.read()
                    conn.sendall(contents.encode())
                    
            elif (data == "BYE"):
                print("Received BYE command from client")
                conn.sendall("Bye".encode())
                flag = 1
                break
                
            elif (data == "TIME"):
                print("Received TIME command from client")
                time = datetime.datetime.now()
                conn.sendall(str(time).encode())
                
            else:
                print("Client issued an unknown command")
                conn.sendall("Command not found".encode())
                 
        if flag == 1:
            conn.close()
            print("Closing connection")
            break
    
    # Adición de excepciones para errores observados
    except ConnectionResetError:
        print("Lost connection")
        break
    except ConnectionAbortedError:
        print("Lost connection")
        break
        
    # En caso de no encontrar el fichero pedido por el cliente, se informa y continua
    # el programa
    except FileNotFoundError:
        print("File requested by client not found")
        conn.sendall("File not found".encode())
        continue


Waiting for clients...
Connection received from client ('127.0.0.1', 56513)
Client issued an unknown command
Received GET command from client
File requested by client not found
Waiting for clients...
Connection received from client ('127.0.0.1', 56531)
Client issued an unknown command
Received GET command from client
Received BYE command from client
Closing connection


### PRÁCTICA TCP 2.2
Modificación del programa anterior para poder modificar el fichero del servidor desde el cliente. Se incluyen dos instrucciones nuevas, LOCK y PUT

In [2]:
import datetime
import socket
import sys

MTU = 1024
flag = 0

# Diccionario de confirmación de bloqueo. En el caso de quererse escalar esto, sería necesaria
# la implementación de un código capaz de escanear en busca de los ficheros .txt en el servidor,
# cuyos nombres serían incluidos en este diccionario.
# - 0: fichero no bloqueado
# - (IP, host): fichero bloqueado por el cliente identificado con (IP, Host)
blocked_files = {"data.txt":0}

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.bind(("127.0.0.1",6780))
sock.listen(1)

while True:
    print("Waiting for clients...")
    conn, client_address = sock.accept()
    print("Connection received from client " + str(client_address))
    
    try:
        while True:
            data = str(conn.recv(MTU).decode())

            if (data == "BYE"):
                print("Received BYE command from client")
                conn.sendall("Bye".encode())
                flag = 1
                break
                
            elif (data[0:3] == "GET"):
                print("Received GET command from client")
                
                filename = data[4:]
                with open(filename) as f:
                    contents = f.read()
                    conn.sendall(contents.encode())
            
            # Al recibir el comando LOCK , el servidor extrae el nombre del archivo a bloquear,
            # comprueba en el diccionario, empleando como clave el nombre del fichero el valor de bloqueo,
            # si es igual a 0, procede a completar el comando e informar al cliente del correcto bloqueo del archivo.
            # Si estuviese bloqueado (!= 0) se informa al cliente.
            elif (data[0:4] == "LOCK"):
                print("Received LOCK command from client")
                
                filename = data[5:]
                if (blocked_files.get(filename) == 0):
                    blocked_files[filename] = client_address
                    conn.sendall("SUCCESS: File has been asociated with your address".encode())
                else:
                    conn.sendall("ERROR: File is blocked".encode())
            
            # El comando PUT se recibiría en el lado del servidor como un string muy largo con las
            # modificaciones en el fichero. En primer lugar se comprueba si el cliente que está enviando
            # esta información es quien ha bloqueado el fichero previamente, si es así, se aceptan e implementan
            # los cambios en el fichero y se desbloquea el fichero (poniendo su valor a 0)
            # Si no fuese así, el servidor informa al cliente de que el archivo está bloqueado por otra persona
            else:
                print("Received PUT command from client")
                
                if (blocked_files.get("data.txt") == client_address):
                    with open("data.txt", 'w') as f:
                        f.write(data)
                        conn.sendall("File has been modified".encode())
                    
                    blocked_files["data.txt"] = 0
                      
                else:
                    conn.sendall("File is blocked".encode())
                 
        if flag == 1:
            conn.close()
            print("Closing connection")
            break
            
    except ConnectionResetError:
        print("Lost connection")
        break
    except ConnectionAbortedError:
        print("Lost connection")
        break
    except FileNotFoundError:
        print("File requested by client not found")
        conn.sendall("File not found".encode())
        continue


Waiting for clients...
Connection received from client ('127.0.0.1', 56541)
Received LOCK command from client
Received PUT command from client
Received BYE command from client
Closing connection
